![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)



<p><font size="5" color='grey'> <b>
MCP - Model Context Protocol
</b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul

from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages, mermaid
from genai_lib.mcp_modul import (
    handle_mcp_request, connect_to_server, initialize_server_connection,
    discover_server_tools, call_server_tool, setup_assistant_mcp_connection,
    process_user_query, get_assistant_status, get_server_info, get_available_tools
)
import json # Wird für die manuelle JSON-Verarbeitung in Schritt 7 benötigt

setup_api_keys(['OPENAI_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()


# 1 | Intro

-----

<p><font color='black' size="5">
Was ist MCP?
</font></p>



Das Model Context Protocol (MCP) ist ein **offenes Protokoll**, das die Kommunikation zwischen Large Language Models (LLMs) und externen Datenquellen bzw. Tools **standardisiert**. Hier sind die drei Hauptkomponenten und ihre Interaktionen:

1.  LLM (Large Language Model)

    Das eigentliche KI-Modell (z.B. gpt-4o-mini oder Claude)
    Verarbeitet Anfragen und generiert Antworten
    Entscheidet, wann externe Ressourcen benötigt werden

2.  MCP Client

    Läuft in der Host-Anwendung (z.B. Claude Desktop, IDEs)
    Verbindet das LLM mit MCP-Servern
    Verwaltet mehrere Server-Verbindungen gleichzeitig
    Übersetzt zwischen LLM-Anfragen und MCP-Protokoll

3.  MCP Server

    Stellt spezifische Funktionalitäten bereit
    Kann verschiedene Ressourcen anbieten:

    Resources: Strukturierte Daten (Dateien, Datenbanken)
    Prompts: Vordefinierte Prompt-Templates
    Tools: Ausführbare Funktionen

**Wichtige Ressourcen:**

[Anthropic MCP](https://www.anthropic.com/news/model-context-protocol)

[OpenAI MCP](https://openai.github.io/openai-agents-python/mcp/)

[MCPServer](https://github.com/modelcontextprotocol/servers)



<p><font color='black' size="5">
Warum wurde MCP entwickelt?
</font></p>

Die Notwendigkeit von MCP ergibt sich aus den Herausforderungen aktueller **KI-API-Interaktionen**. Derzeit ist der Aufbau von KI-Agenten, die Daten aus verschiedenen Quellen abrufen, **fragmentiert, repetitiv und schwer zu skalieren**. Jedes Tool spricht seine eigene Sprache und erfordert **individuelle Integrationen**. MCP zielt darauf ab, diese Komplexität zu reduzieren und den **Entwicklungsaufwand zu minimieren**.



<p><font color='black' size="5">
MCP-Architektur
</font></p>




MCP verwendet eine Client-Server-Architektur mit den Komponenten:

1.  **LLM (Large Language Model)**

    + Das eigentliche KI-Modell (z.B. gpt-4o-mini oder Claude)
    + Verarbeitet Anfragen und generiert Antworten
    + Signalisiert im generierten Output, wenn externe Ressourcen benötigt werden

2.  **MCP Client**

    + Läuft zusammen mit dem LLM in der **Host-Anwendung** (z.B. Claude Desktop, IDEs)
    + Verbindet das LLM mit MCP-Servern
    + Verwaltet mehrere Server-Verbindungen gleichzeitig
    + Übersetzt zwischen LLM-Anfragen und MCP-Protokoll

    **Hinweis:** LLM und Client bilden gemeinsam die Host-Anwendung.

3.  **MCP Server**

    + Stellt spezifische Funktionalitäten bereit
    + Kann verschiedene Ressourcen anbieten:

        - **Resources**: Strukturierte Daten (Dateien, Datenbanken)
        - **Prompts**: Vordefinierte Prompt-Templates
        - **Tools**: Ausführbare Funktionen



**Kommunikationsfluss:**

AI-Assistent → MCP-Client → MCP-Server → Externe Ressource
                    ↓            ↓            ↓
    Antwort    ←  JSON/HTTP  ←  Daten-API  ←  Raw Data



<p><font color='black' size="5">
Praktische Implementierung
</font></p>



Die Implementierung von MCP erfolgt schrittweise:

**1. Server-Konfiguration**

  - Definition der verfügbaren Ressourcen
  - Sicherheitsrichtlinien festlegen
  - Authentifizierung konfigurieren

**2. Client-Integration**

  - Verbindung zum MCP-Server herstellen
  - Verfügbare Tools und Datenquellen erkunden
  - Anfragen formulieren und Antworten verarbeiten

**3. Testing und Deployment**

  - Funktionalitätstests durchführen
  - Performance optimieren
  - Produktionsumgebung einrichten


<p><font color='black' size="5">
MCP vs. andere Ansätze
</font></p>

**MCP vs. Function Calling - Der entscheidende Unterschied:**

Der Hauptunterschied liegt in der **Architektur und Standardisierung**:

**Function Calling (z.B. OpenAI):**

```python
# Direkte Integration im AI-System
tools = [
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads a file",
            "parameters": {"type": "object", "properties": {...}}
        }
    }
]
response = openai.chat.completions.create(model="gpt-4", tools=tools)
```

**MCP (Model Context Protocol):**

```python
# Standardisierte Server-Client-Architektur
request = {
    "jsonrpc": "2.0",
    "method": "tools/call",
    "params": {"name": "read_file", "arguments": {...}}
}
response = await handle_mcp_request(request)
```

**Detaillierter Vergleich:**

| Aspekt | MCP | Function Calling | RAG | Custom APIs |
|--------|-----|------------------|-----|-------------|
| **Architektur** | Client-Server | Direkt integriert | Pipeline-basiert | Individuell |
| **Standardisierung** | ✅ Offenes Protokoll | ❌ Modell-spezifisch | ❌ Framework-abhängig | ❌ Individuell |
| **Wiederverwendbarkeit** | ✅ Server für alle AIs | ❌ Pro AI-System | ⚠️ Begrenzt | ❓ Variabel |
| **Setup-Aufwand** | 🟡 Moderat | 🟢 Niedrig | 🟡 Moderat | 🔴 Hoch |
| **Enterprise-Ready** | ✅ Vollständig | ⚠️ Begrenzt | 🟡 Teilweise | ❓ Variabel |
| **Sicherheit** | ✅ Integrierte Kontrollen | ⚠️ Basic | ⚠️ Dokumenten-basiert | ❓ Variabel |
| **Bidirektional** | ✅ Lesen & Schreiben | ✅ Ja | ❌ Nur Lesen | ✅ Ja |
| **Echtzeit** | ✅ Live-Daten | ✅ Ja | ❌ Statische Docs | ✅ Ja |
| **Skalierung** | ✅ Einfach | 🔴 Schwierig | 🟡 Moderat | ❓ Variabel |

**Praktische Entscheidungshilfe:**

**Function Calling verwenden für:**

  - ✅ Prototyping und schnelle Tests
  - ✅ Einfache, direkte Tool-Integration
  - ✅ Ein AI-System mit wenigen Tools
  - ✅ Schneller Entwicklungsstart

**MCP verwenden für:**

  - ✅ Enterprise-Anwendungen
  - ✅ Mehrere AI-Systeme mit geteilten Tools
  - ✅ Sicherheitskritische Umgebungen
  - ✅ Langfristige, skalierbare Architekturen
  - ✅ Cross-Platform-Kompatibilität

**Fazit:** Function Calling ist der einfacherer Einstieg, MCP ist die zukunftssichere Enterprise-Lösung.

# 2 | Modul `mcp`
---

Das `mcp_modul.py` bietet eine **funktionale All-in-One-Lösung** zur Demonstration der MCP-Architektur mit klarer Rollentrennung zwischen **LLM**, **Client** und **Server**.

<p><font color='black' size="5">
Kommunikationsfluss
</font></p>

```
Benutzer
   ↓
LLM (versteht Anfrage, entscheidet: "Ich brauche list_files")
   ↓
LLM (parsed eigenen Tool-Intent aus generiertem Text)
   ↓
Client (übersetzt in MCP-Request, sendet an Server)
   ↓
Server (führt list_files aus)
   ↓
Client (empfängt Ergebnis)
   ↓
LLM (synthetisiert Antwort aus Ergebnis)
   ↓
Benutzer
```


<p><font color='black' size="5">
Kernfunktionen nach Rolle
</font></p>

[MCP Simulation](https://editor.p5js.org/ralf.bendig.rb/full/CkYSKZe8t)

<p><font color='black' size="5">
8 Schritte der MCP-Simulation
</font></p>



Die MCP-Demo zeigt den vollständigen Workflow in 8 Schritten:

**Schritte 1-5: Verbindungsaufbau (Setup-Phase)**

1. **Client → Server**: Verbindung herstellen (`connect_to_server`)
2. **Client → Server**: INIT-Request senden (`initialize_server_connection`)
3. **Server → Client**: INIT-Response mit Server-Info
4. **Client → Server**: TOOLS/LIST-Request senden (`discover_server_tools`)
5. **Server → Client**: TOOLS/LIST-Response mit verfügbaren Tools

**Schritte 6-8: Task-Ausführung (Query-Phase)**

6. **LLM → Client → Server**: LLM analysiert Query, Client sendet TOOLS/CALL-Request (`call_server_tool`)
7. **Server → Client**: TOOLS/CALL-Response mit Tool-Ergebnis (JSON-Payload)
8. **LLM**: LLM synthetisiert finale natürlichsprachliche Antwort aus Tool-Ergebnis



<p><font color='black' size="5">
Kernfunktionen nach Schritten
</font></p>



**Setup-Phase (Schritte 1-5):**

  - `connect_to_server()`: Schritt 1 - Logische Verbindung herstellen
  - `initialize_server_connection()`: Schritte 2-3 - Server initialisieren
  - `discover_server_tools()`: Schritte 4-5 - Verfügbare Tools entdecken
  - `setup_assistant_mcp_connection()`: Wrapper für vollständigen Setup

**Query-Phase (Schritte 6-8):**

  - `call_server_tool()`: Schritt 6 - Tool-Call Request senden
  - `handle_mcp_request()`: Schritt 7 - Server verarbeitet Request und sendet Response
  - LLM-Synthese: Schritt 8 - Finale Antwort generieren (in Code manuell implementiert)

**Server-Komponenten:**

  - `handle_mcp_request()`: Verarbeitet alle MCP-Requests (JSON-RPC 2.0)
  - `register_new_tool()`: Dynamische Tool-Registrierung zur Laufzeit
  - **Tools**: `read_file_tool`, `write_file_tool`, `list_files_tool`, `get_system_info_tool`

<p><font color='black' size="5">
Kommunikationskonvention
</font></p>

**Tool-Intent-Syntax**

Das LLM signalisiert seine Tool-Absichten durch eine spezielle Syntax im generierten Text:

```
[MCP_CALL: <tool_name>({<arguments_json>})] [/MCP_CALL]
```

**Beispiel:**

```
[MCP_CALL: read_file({"path": "/projekt/data.txt"})] [/MCP_CALL]
```

Diese Syntax wird von der LLM-Komponente selbst erkannt und dann über den Client in MCP-konforme JSON-RPC 2.0 Requests übersetzt.

**JSON-RPC Protokoll**

**JSON-RPC** (JSON Remote Procedure Call) ist ein leichtgewichtiges Protokoll für entfernte Funktionsaufrufe. Es ermöglicht dem Client, Tools auf dem Server aufzurufen, als wären sie lokale Funktionen.

**Beispiel eines JSON-RPC Requests:**

```json
{
  "jsonrpc": "2.0",
  "method": "tools/call",
  "params": {
    "name": "read_file",
    "arguments": {"path": "/projekt/data.txt"}
  },
  "id": 1
}
```

JSON-RPC nutzt JSON für strukturierte Nachrichten und definiert klare Regeln für Requests, Responses und Fehlerbehandlung über Netzwerk- oder Prozessgrenzen hinweg.


<p><font color='black' size="5">

Funktionsumfang   `mcp.py`  

</font></p>



Das mcp\_modul.py ist eine funktionale Demonstration der MCP-Architektur, die echte und simulierte Elemente kombiniert:

| Komponente | Demo-Implementierung | Production-System |
|------------|---------------------|-------------------|
| **Tool-Ausführung** | ✅ Real (echte Datei-Ops) | ✅ Real |
| **LLM-Calls** | ✅ Real (OpenAI API) | ✅ Real |
| **JSON-RPC Format** | ✅ Real (Standard-konform) | ✅ Real |
| **Netzwerk-Transport** | ❌ Simuliert (In-Memory) | ✅ Real (stdio/TCP) |
| **Prozess-Trennung** | ❌ Simuliert (ein Prozess) | ✅ Real (separate Prozesse) |
| **Connection Setup** | ❌ Simuliert (Dictionary) | ✅ Real (Prozess-Management) |
| **Async I/O** | ✅ Real (asyncio) | ✅ Real |

Die Demo ist ideal für:

✅ Lernen: Verstehen der MCP-Konzepte ohne Komplexität von Prozess-Management    
✅ Prototyping: Schnelles Testen von Tool-Implementierungen    
✅ Testing: Einfaches Unit-Testing ohne Setup von separaten Prozessen    


<p><font color='black' size="5">
MCP-Modul importieren
</font></p>

In [ ]:
from genai_lib.mcp_modul import (
    # Server-Funktionen
    handle_mcp_request,
    get_server_info,
    register_new_tool,

    # Client-Funktionen
    setup_full_connection,
    call_server_tool,
    get_available_tools,
    get_client_status,

    # Assistant-Funktionen
    setup_assistant_mcp_connection,
    process_user_query,
    get_assistant_status,

    # Modul-Info
    get_module_info,
    connect_to_server # Explizit für Schritt 1 der Simulation
)

# 3 | MCP-Server erstellen

-----

Ein MCP-Server stellt die Verbindung zwischen AI-Assistenten und externen Ressourcen her. Wir erstellen einen funktionalen Server, der Datei-Operationen und Systemdaten bereitstellt.

In [ ]:
#@title 🔧 Server-Status prüfen  { display-mode: "form" }

# Test des MCP-Servers (vereinfacht mit Modul)
async def test_mcp_server():
    """Testet die grundlegenden Server-Funktionen"""

    mprint("### 🧪 MCP-Server Statusanfrage ...\n")
    mprint("---")

    # 1. Server-Info
    print("1️⃣ Server-Informationen:")
    info = get_server_info()
    print(f"   Name: {info['name']}")
    print(f"   Version: {info['version']}")
    print(f"   Tools: {', '.join(info['available_tools'])}\n")

    # 2. Tools auflisten
    tools_request = {
        "jsonrpc": "2.0",
        "id": "tools-1",
        "method": "tools/list"
    }

    response = await handle_mcp_request(tools_request)
    print("2️⃣ Tool-Liste:")
    for tool in response['result']['tools']:
        print(f"   • {tool['name']}: {tool['description']}")
    print()

    # 3. Systeminformationen abrufen
    sysinfo_request = {
        "jsonrpc": "2.0",
        "id": "call-1",
        "method": "tools/call",
        "params": {
            "name": "get_system_info",
            "arguments": {}
        }
    }

    response = await handle_mcp_request(sysinfo_request)
    result = json.loads(response["result"]["content"][0]["text"])
    print("3️⃣ Systeminformationen:")
    print(f"   System: {result['system']}")
    print(f"   Python: {result['python_version']}\n")

    print("✅ MCP-Server erfolgreich angefragt!")

# Tests ausführen
await test_mcp_server()

# 4 | MCP-Demo

-----

In [ ]:
#@title 🤖 AI-Assistant mit MCP-Server verbinden (SCHRITTE 1-5 der Simulation) { display-mode: "form" }

SERVER_NAME = "file-server"
server_handler = handle_mcp_request

mprint("### 🛠️ Simuliertes MCP-Setup (Schritte 1-5)")
mprint("------------------------------------------")

# 1. SCHRITT 1: Verbindung herstellen
print("1️⃣ SCHRITT 1: Client versucht, Verbindung herzustellen...")
connect_to_server(SERVER_NAME, server_handler)
print("✅ Client ist logisch VERBUNDEN.")


# 2. SCHRITT 2 & 3: Initialisierung (Request & Response)
mprint("\n2️⃣ & 3️⃣ SCHRITT 2/3: Initialisierung (INIT-Request & Response):")
# initialize_server_connection sendet Request (2) und erwartet Response (3)
init_response = await initialize_server_connection(SERVER_NAME)
print(f"   INIT-Response erhalten (Schritt 3). Status: {init_response.get('result', {}).get('serverInfo', {}).get('version')}")


# 3. SCHRITT 4 & 5: Tools entdecken (Request & Response)
mprint("\n4️⃣ & 5️⃣ SCHRITT 4/5: Tools entdecken (Discovery Request & Response):")
# discover_server_tools sendet Request (4) und erwartet Response (5)
tools_list = await discover_server_tools(SERVER_NAME)
print(f"   ✅ {len(tools_list)} Tools entdeckt und registriert (Schritt 5).")

# 4. Assistant mit MCP-Server verbinden (Finaler Setup-Schritt im Assistenten)
result = setup_assistant_mcp_connection(SERVER_NAME)
mprint(f"\n🎯 FINAL: Assistant Setup Ergebnis: {result}\n")

In [ ]:
#@title 🤖 Verbindungsstatus prüfen { display-mode: "form" }
# Status prüfen
mprint("### ✅ Verbindungs-Status")
mprint("---")

status = get_assistant_status()
print("🤖 Assistant-Konfiguration:")
print(f"   Name: {status['config']['name']}")
print(f"   Modell: {status['config']['openai_model']}")
print(f"   MCP aktiviert: {status['mcp_enabled']}")
print(f"   Verbundener Server: {status['connected_server']}")
print(f"   Verfügbare Server: {status['available_servers']}\n")

print("🛠️ Verfügbare Tools:")
for server, tool_list in status['available_tools'].items():
    print(f"   Server '{server}':")
    for tool in tool_list:
        print(f"      • {tool}")

print("\n💡 Assistant ist bereit für Anfragen!")

In [ ]:
#@title 🤖 Query 1: Was für ein System läuft hier? (Schritte 6-8) { display-mode: "form" }

# Dieser Code zerlegt den Task-Workflow explizit in die drei finalen Schritte
# der MCP-Simulation (6-8), anstatt die Blackbox-Funktion process_user_query zu nutzen.

query_1 = "Was für ein System läuft hier?"
TOOL_NAME = "get_system_info" # Das vom LLM gewählte Tool für die Anfrage
TOOL_ARGS = {}

mprint(f"\n#### Query 1: {query_1}")
print("\n🟢 START: LLM-Orchestrierung (zerlegt)")
print("-" * 40)

# SCHRITT 6: LLM-Analyse und Tool-Call (Request)
mprint(f"6️⃣ SCHRITT 6: LLM sendet TOOLS/CALL-Request für Tool '{TOOL_NAME}'")
# Die Funktion 'call_server_tool' sendet den Request und wartet auf die Antwort
tool_call_response = await call_server_tool(
    server_name=SERVER_NAME,
    tool_name=TOOL_NAME,
    arguments=TOOL_ARGS
)

# SCHRITT 7: Server-Antwort (Response)
mprint("\n7️⃣ SCHRITT 7: Server-Antwort (Tool-Ergebnis) erhalten:")

# Manuelle Extraktion des Tool-Ergebnisses (simuliert die Verarbeitung des Clients)
if tool_call_response.get("result"):
    result_content = tool_call_response["result"]["content"][0]["text"]
    result_data = json.loads(result_content)
    mprint(f"   Datenbasis für Synthese: System läuft auf {result_data.get('system')} (Payload von Server)")

    # SCHRITT 8: LLM generiert finale Antwort (Synthese)
    mprint("\n8️⃣ SCHRITT 8: LLM generiert finale Antwort (Synthese):")
    final_answer = (
        f"Hier läuft ein **{result_data.get('system', 'unbekanntes')}** System "
        f"mit einer Python Version **{result_data.get('python_version', 'unbekannt')}**."
    )
    mprint(f"🤖 {final_answer}")
else:
    mprint("❌ Tool-Ausführung fehlgeschlagen. Keine finale Antwort möglich.")

print("\n🔴 ENDE: Zerlegte Sequenz abgeschlossen.")

In [ ]:
#@title 🤖 Query 2: Erstelle eine Datei 'demo.txt' (Schritte 6-8) { display-mode: "form" }

# Dieser Code zerlegt den Task-Workflow explizit in die drei finalen Schritte
# der MCP-Simulation (6-8), anstatt die Blackbox-Funktion process_user_query zu nutzen.

query_2 = "Erstelle eine Datei 'demo.txt' mit dem Inhalt 'Hello MCP!'"
TOOL_NAME = "write_file" # Das vom LLM gewählte Tool für die Anfrage
TOOL_ARGS = {"filepath": "demo.txt", "content": "Hello MCP!"}

mprint(f"\n#### Query 2: {query_2}")
print("\n🟢 START: LLM-Orchestrierung (zerlegt)")
print("-" * 40)

# SCHRITT 6: LLM-Analyse und Tool-Call (Request)
mprint(f"6️⃣ SCHRITT 6: LLM sendet TOOLS/CALL-Request für Tool '{TOOL_NAME}'")
# Die Funktion 'call_server_tool' sendet den Request und wartet auf die Antwort
tool_call_response = await call_server_tool(
    server_name=SERVER_NAME,
    tool_name=TOOL_NAME,
    arguments=TOOL_ARGS
)

# SCHRITT 7: Server-Antwort (Response)
mprint("\n7️⃣ SCHRITT 7: Server-Antwort (Tool-Ergebnis) erhalten:")

# Manuelle Extraktion des Tool-Ergebnisses (simuliert die Verarbeitung des Clients)
if tool_call_response.get("result"):
    result_content = tool_call_response["result"]["content"][0]["text"]
    result_data = json.loads(result_content)
    mprint(f"   Datenbasis für Synthese: {result_data.get('message', 'Datei erstellt')} (Payload von Server)")

    # SCHRITT 8: LLM generiert finale Antwort (Synthese)
    mprint("\n8️⃣ SCHRITT 8: LLM generiert finale Antwort (Synthese):")
    final_answer = (
        f"Ich habe die Datei **demo.txt** erfolgreich erstellt mit dem Inhalt: **Hello MCP!**"
    )
    mprint(f"🤖 {final_answer}")
else:
    mprint("❌ Tool-Ausführung fehlgeschlagen. Keine finale Antwort möglich.")

print("\n🔴 ENDE: Zerlegte Sequenz abgeschlossen.")

In [ ]:
#@title 🤖 Query 3: Welche Dateien sind im aktuellen Verzeichnis? (Schritte 6-8) { display-mode: "form" }

# Dieser Code zerlegt den Task-Workflow explizit in die drei finalen Schritte
# der MCP-Simulation (6-8), anstatt die Blackbox-Funktion process_user_query zu nutzen.

query_3 = "Welche Dateien sind im aktuellen Verzeichnis?"
TOOL_NAME = "list_files" # Das vom LLM gewählte Tool für die Anfrage
TOOL_ARGS = {}

mprint(f"\n#### Query 3: {query_3}")
print("\n🟢 START: LLM-Orchestrierung (zerlegt)")
print("-" * 40)

# SCHRITT 6: LLM-Analyse und Tool-Call (Request)
mprint(f"6️⃣ SCHRITT 6: LLM sendet TOOLS/CALL-Request für Tool '{TOOL_NAME}'")
# Die Funktion 'call_server_tool' sendet den Request und wartet auf die Antwort
tool_call_response = await call_server_tool(
    server_name=SERVER_NAME,
    tool_name=TOOL_NAME,
    arguments=TOOL_ARGS
)

# SCHRITT 7: Server-Antwort (Response)
mprint("\n7️⃣ SCHRITT 7: Server-Antwort (Tool-Ergebnis) erhalten:")

# Manuelle Extraktion des Tool-Ergebnisses (simuliert die Verarbeitung des Clients)
if tool_call_response.get("result"):
    result_content = tool_call_response["result"]["content"][0]["text"]
    result_data = json.loads(result_content)
    files_list = result_data.get('files', [])
    mprint(f"   Datenbasis für Synthese: {len(files_list)} Dateien gefunden (Payload von Server)")

    # SCHRITT 8: LLM generiert finale Antwort (Synthese)
    mprint("\n8️⃣ SCHRITT 8: LLM generiert finale Antwort (Synthese):")
    if files_list:
        files_formatted = ', '.join([f"**{f}**" for f in files_list])
        final_answer = f"Im aktuellen Verzeichnis befinden sich folgende Dateien: {files_formatted}"
    else:
        final_answer = "Das aktuelle Verzeichnis ist leer."
    mprint(f"🤖 {final_answer}")
else:
    mprint("❌ Tool-Ausführung fehlgeschlagen. Keine finale Antwort möglich.")

print("\n🔴 ENDE: Zerlegte Sequenz abgeschlossen.")

# 5 | MCP in der Praxis
---

<p><font color='black' size="5">
Anwendungsfälle
</font></p>



MCP bietet in Unternehmensumgebungen erhebliche Vorteile, besonders wenn funktional implementiert:

**1. Datenbank-Integration**

  - Direkter Zugriff auf Unternehmens-Datenbanken
  - Echtzeit-Berichte und Analytics
  - Automatisierte Datenextraktion

**2. ERP-System-Anbindung**

  - Zugriff auf SAP, Oracle, Microsoft Dynamics
  - Automatisierte Bestellprozesse
  - Inventory-Management

**3. Cloud-Service-Integration**

  - AWS, Azure, Google Cloud APIs
  - Automatisierte Ressourcenverwaltung
  - Monitoring und Alerting

**4. Sicherheits-Tools**

  - SIEM-System-Integration
  - Automatisierte Incident Response
  - Compliance-Reporting

**5. Collaboration-Tools**

  - Slack, Teams, Jira Integration
  - Automatisierte Projekt-Updates
  - Team-Koordination

**Vorteile des funktionalen Ansatzes:**

  - **Bessere Testbarkeit**: Einzelne Funktionen können isoliert getestet werden
  - **Flexiblere Komposition**: Tools können dynamisch kombiniert werden
  - **Einfachere Wartung**: Klare Trennung von Concerns
  - **Bessere Performance**: Keine Overhead durch Objekt-Instantiierung


# 6 | Zukunft von MCP
---



**Entwicklungstrends:**

**1. Erweiterte Tool-Kategorien**

  - Computer Vision APIs
  - IoT-Device Integration
  - Blockchain/Smart Contract Tools
  - Robotics Control Systems

**2. Cross-Platform Standardisierung**

  - OpenAPI/JSON Schema Integration
  - GraphQL-basierte Schemas
  - Universelle Authentication
  - Multi-Cloud Orchestrierung

**3. Enterprise-Integration**

  - Native SAP/Oracle Connectors
  - Compliance-by-Design
  - Zero-Trust Security Models
  - Hybrid Cloud Architectures

**Ausblick:**
MCP entwickelt sich zu einem universellen Standard für AI-System-Integration. Der funktionale Ansatz bietet dabei besondere Vorteile:

  - **Bessere Komposierbarkeit**: Funktionen können flexibel kombiniert werden
  - **Einfachere Tests**: Isolierte Funktionen sind leichter zu testen
  - **Klarere Architektur**: Funktionale Programmierung fördert saubere Designs
  - **Bessere Performance**: Weniger Overhead durch Objekt-Management

In den nächsten Jahren erwarten wir:

  - **Standardisierung** durch W3C oder ähnliche Organisationen
  - **Native Support** in allen großen AI-Plattformen
  - **Enterprise-Grade** Sicherheit und Compliance
  - **Ökosystem** von vordefinierten funktionalen Connectoren
  - **No-Code/Low-Code** MCP-Integration Tools
  - **Funktionale Komposition** als Standard-Architektur

MCP wird die Art und Weise revolutionieren, wie AI-Systeme mit der realen Welt interagieren, und der funktionale Ansatz wird dabei eine Schlüsselrolle spielen.


# A | Aufgabe
---

<p><font color='black' size="5">
Teste den AI-Assistant mit eigenen Queries
</font></p>



**Ziel**: Nutze den fertigen MCP-Assistant, um verschiedene Datei-Operationen durchzuführen.

**Schritt 1: Einfache Queries testen**

Führe die folgenden Queries mit dem AI-Assistant aus und beobachte, wie er die MCP-Tools nutzt:

In [ ]:
# Query 1: System-Informationen
result = await process_user_query("Welches Betriebssystem läuft hier?")
print(result)

# Query 2: Datei erstellen
result = await process_user_query("Erstelle eine Datei 'meine_notizen.txt' mit dem Text 'Das ist mein MCP-Test'")
print(result)

# Query 3: Datei lesen
result = await process_user_query("Lies die Datei 'meine_notizen.txt' und zeige mir den Inhalt")
print(result)

# Query 4: Verzeichnis auflisten
result = await process_user_query("Welche Dateien sind im aktuellen Verzeichnis?")
print(result)

**Schritt 2: Eigene Queries erstellen**

Entwickle mindestens **3 eigene Queries**, die verschiedene Tool-Kombinationen nutzen:

**Beispiele für kreative Queries:**

  - "Erstelle eine ToDo-Liste in der Datei 'todos.txt' mit 5 Aufgaben"
  - "Zähle, wie viele .txt Dateien im aktuellen Verzeichnis sind"
  - "Erstelle eine Datei mit den aktuellen Systeminformationen"
  - "Lies alle .txt Dateien und fasse ihren Inhalt zusammen"

**Schritt 3: Bonus-Challenge (Optional)**

Teste den Assistant mit und ohne MCP-Modus und vergleiche die Ergebnisse:

In [ ]:
# Mit MCP
result_mit = await process_user_query("Was für Dateien sind hier?", use_mcp=True)

# Ohne MCP
result_ohne = await process_user_query("Was für Dateien sind hier?", use_mcp=False)

# Vergleiche
print("MIT MCP:", result_mit)

print("\nOHNE MCP:", result_ohne)

**💡 Tipps:**

  - Der Assistant interpretiert deine natürlichsprachlichen Anfragen
  - Er entscheidet selbst, welche MCP-Tools er nutzt
  - Experimentiere mit verschiedenen Formulierungen
  - Achte auf die Console-Ausgabe, um zu sehen, welche Tools aufgerufen werden

**✅ Erfolg:**

Du hast die Aufgabe erfolgreich abgeschlossen, wenn:

  - ✅ Mindestens 3 eigene Queries erstellt
  - ✅ Alle Queries liefern sinnvolle Ergebnisse
  - ✅ Du verstehst, wie der Assistant MCP-Tools nutzt

<!-- end list -->

```
```